# Agentic RAG with LlamaIndex


In this notebook we will experiment RAG with multi-document agent.

- Define a reader to read the `pdf` sample file [AraGPT2](./data/aragpt2.pdf) paper.
- Define a `splitter` to process the texts of the document.
- Set the LLM embedding and generation model ids.
- Create the engines from the Indexes and define a tool wrapper around them.
- Create Index for tool objects.
- Define the agent worker and agent runner that utilize memory.
- Excute the multi-docs agent.


## Setups


In [1]:
from rich import print
from dotenv import load_dotenv

In [2]:
# load env variables
_ = load_dotenv()

In [3]:
# define some constants
GENERATION_MODEL_ID = "gpt-4o-mini"
EMBEDDING_MODEL_ID = "text-embedding-3-small"

## Load Documents


In [16]:
from llama_index.core.schema import TextNode
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter



def get_nodes(file_paths: list[str]) -> dict[str, TextNode]:
    """Extract text nodes from documents.
    
    inputs:
        file_paths (list[str]): paths to pdf files. must be unique.
    returns:
        nodes_dict (dict[str, TextNode]): mapping of file paths to nodes.
    """
    nodes_dict = {file_path.split("/")[-1].split(".")[0]: [] for file_path in file_paths}
    documents_reader = SimpleDirectoryReader(input_files=file_paths)
    documents = documents_reader.load_data()
    sentence_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=64)
    nodes = sentence_splitter.get_nodes_from_documents(documents)
    for node in nodes:
        nodes_dict[node.metadata["file_name"].split(".")[0]].append(node)
    return nodes_dict

In [17]:
import glob

file_paths = glob.glob("data/*")
print(file_paths)

['data/arabert.pdf', 'data/gpt2.pdf', 'data/aragpt2.pdf', 'data/camel_parser.pdf', 'data/camel_bert.pdf']

In [18]:
nodes = get_nodes(file_paths=file_paths)

In [20]:
print(nodes.keys())

dict_keys(['arabert', 'gpt2', 'aragpt2', 'camel_parser', 'camel_bert'])

## Define Vector Search and Summary Tools


In [21]:
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata


def get_tools_from_nodes(
    nodes: dict[str, TextNode]
) -> dict[str, list[QueryEngineTool]]:
    """Define query engine tools from nodes dictionary.

    inputs:
        nodes (dict[str, TextNode]): text nodes for each document by name.
    returns:
        tools_dict (dict[str, list[QueryEngineTool]]): tools for each document by name.
    """
    tools_dict = {file_name: [] for file_name in nodes.keys()}
    for file_name, text_nodes in nodes.items():
        print(f"Creating tools for file: {file_name}")
        # define vector tool
        vector_index = VectorStoreIndex(nodes=text_nodes)
        vector_engine = vector_index.as_query_engine()
        vector_metadata = ToolMetadata(
            name=f"vector_tool_for_{file_name}",
            description=f"Useful for retrieving specific context from the {file_name} paper.",
        )
        vector_tool = QueryEngineTool(
            query_engine=vector_engine, metadata=vector_metadata
        )
        # define summary tool
        summary_index = SummaryIndex(nodes=text_nodes)
        summary_engine = summary_index.as_query_engine(response_mode="tree_summarize")
        summary_metadata = ToolMetadata(
            name=f"summary_tool_for_{file_name}",
            description=f"Useful for summarization questions related to the {file_name} paper.",
        )
        summary_tool = QueryEngineTool(
            query_engine=summary_engine, metadata=summary_metadata
        )
        tools_dict[file_name].extend([vector_tool, summary_tool])
    return tools_dict

In [22]:
tools = get_tools_from_nodes(nodes)

Creating tools for file: arabert

Creating tools for file: gpt2

Creating tools for file: aragpt2

Creating tools for file: camel_parser

Creating tools for file: camel_bert

In [23]:
print(tools["arabert"][0])

<llama_index.core.tools.query_engine.QueryEngineTool object at 0x7fe9f4ab9d50>

## Define Objects Vector Search Tool

This will do vector search over the tools we have.


In [38]:
from llama_index.core.objects import ObjectIndex, ObjectRetriever


def get_object_tool(tools: dict[str, list[QueryEngineTool]]) -> ObjectRetriever:
    """Define tools query engine tool.

    inputs:
        tools (dict[str, list[QueryEngineTool]]): tools for each document by name.
    returns:
        tools_retriever (ObjectRetriever): tools object retriever.
    """
    tools_index = ObjectIndex.from_objects(
        objects=[tl for _, tls in tools.items() for tl in tls], index_cls=VectorStoreIndex
    )
    tools_retriever = tools_index.as_retriever(similarity_top_k=4)
    return tools_retriever

In [39]:
tools_retriever = get_object_tool(tools)
print(tools_retriever)

<llama_index.core.objects.base.ObjectRetriever object at 0x7fe9f646a9d0>

## Define Agent Worker and Runner


### Setup LLM Backend


In [28]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model=GENERATION_MODEL_ID)
Settings.embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL_ID)

### Define Agent


In [32]:
sys_prompt = """\
You are an agent designed to answer queries over a set of \
given papers. Please always use the tools provided to answer \
a question. Do not rely on prior knowledge.\
"""

print(sys_prompt)

You are an agent designed to answer queries over a set of given papers. Please always use the tools provided to 
answer a question. Do not rely on prior knowledge.

In [40]:
from llama_index.core.agent import (
    AgentRunner,
    FunctionCallingAgentWorker,
)

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=tools_retriever,
    system_prompt=sys_prompt,
    verbose=True,
)

agent = AgentRunner(agent_worker)

In [41]:
query = "Compare between the AraBERT and AraGPT2 models on a high level."


response = agent.chat(query)

Added user message to memory: Compare between the AraBERT and AraGPT2 models on a high level.
=== Calling Function ===
Calling function: vector_tool_for_aragpt2 with args: {"input": "AraGPT2 model overview"}
=== Function Output ===
The AraGPT2 model is the first advanced Arabic language generation model developed from scratch on a large Arabic corpus of internet text and news articles. It comes in four size variants: base, medium, large, and mega, with the mega model having 1.46 billion parameters, making it the largest Arabic language model available. The model has been evaluated successfully on various tasks such as synthetic news generation and zero-shot question answering. Additionally, it achieved a perplexity of 29.8 on held-out Wikipedia articles and demonstrated the ability to generate news articles that are challenging to distinguish from those written by humans. An automatic discriminator model with a 98% accuracy in detecting model-generated text has also been developed and 

In [42]:
print(response.response)

Here's a high-level comparison between the AraBERT and AraGPT2 models:

### AraBERT
- **Purpose**: Focuses on natural language understanding (NLU) in Arabic.
- **Architecture**: Transformer-based model.
- **Training Method**: Pre-trained using the Masked Language Modeling (MLM) task.
- **Applications**: Designed to improve performance in various NLU tasks related to Arabic text.

### AraGPT2
- **Purpose**: Advanced Arabic language generation model.
- **Architecture**: Developed from scratch on a large Arabic corpus of internet text and news articles.
- **Variants**: Comes in four size variants (base, medium, large, and mega), with the mega model having 1.46 
billion parameters.
- **Performance**: Evaluated on tasks like synthetic news generation and zero-shot question answering, achieving a 
perplexity of 29.8 on held-out Wikipedia articles.
- **Human-like Generation**: Demonstrates the ability to generate news articles that are difficult to distinguish 
from human-written content.
- **Discriminator Model**: An automatic discriminator model with 98% accuracy in detecting model-generated text was
developed alongside it.

In summary, AraBERT is tailored for understanding Arabic text, while AraGPT2 is geared towards generating Arabic 
text, showcasing significant capabilities in language generation tasks.